In [91]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import torch.nn.functional as F

In [92]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [93]:
batch_size = 4
num_epochs = 4
learning_rate = .001
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [94]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [95]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data  = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_data,  batch_size=batch_size, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [96]:
example = iter(train_loader)
sample, label = example._next_data()
print(sample.shape)

torch.Size([4, 3, 32, 32])


In [97]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = out.view(-1, 16*5*5)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out



In [98]:
model = ConvNet().to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [99]:
n_total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1)%2000 == 0:
            print(f"epoch {epoch} / {num_epochs}, step {i} / {n_total_step}, loss = {loss.item():.4f}")

epoch 0 / 4, step 1999 / 12500, loss = 2.2329
epoch 0 / 4, step 3999 / 12500, loss = 2.2767
epoch 0 / 4, step 5999 / 12500, loss = 2.1633
epoch 0 / 4, step 7999 / 12500, loss = 1.4770
epoch 0 / 4, step 9999 / 12500, loss = 2.1059
epoch 0 / 4, step 11999 / 12500, loss = 2.1532
epoch 1 / 4, step 1999 / 12500, loss = 1.8937
epoch 1 / 4, step 3999 / 12500, loss = 2.0193
epoch 1 / 4, step 5999 / 12500, loss = 1.7816
epoch 1 / 4, step 7999 / 12500, loss = 1.8061
epoch 1 / 4, step 9999 / 12500, loss = 2.1434
epoch 1 / 4, step 11999 / 12500, loss = 1.4051


KeyboardInterrupt: 

In [103]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)

        _, prediction = torch.max(output, 1)

        n_samples += labels.size(0)

        n_correct += (prediction == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred  = prediction[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f"accuracy:{acc}")

    for i in range(10):
        acc = 100 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}:{acc}')

accuracy:42.38
Accuracy of plane:57.4
Accuracy of car:60.7
Accuracy of bird:18.7
Accuracy of cat:22.7
Accuracy of deer:40.6
Accuracy of dog:30.8
Accuracy of frog:40.2
Accuracy of horse:65.8
Accuracy of ship:52.4
Accuracy of truck:34.5
